In [2]:
# Étape 1 : Extraction des Flux RSS

import feedparser

def recuperer_bulletins_rss(url):
    rss_feed = feedparser.parse(url)
    bulletins = []
    for entry in rss_feed.entries:
        bulletin = {
            "Titre du bulletin (ANSSI)": entry.title,
            "Description": entry.description,
            "Lien du bulletin (ANSSI)": entry.link,
            "Date de publication": entry.published
        }
        bulletins.append(bulletin)
    return bulletins

# URLs des flux RSS
avis_url = "https://www.cert.ssi.gouv.fr/avis/feed/"
alerte_url = "https://www.cert.ssi.gouv.fr/alerte/feed/"

avis_bulletins = recuperer_bulletins_rss(avis_url)
alerte_bulletins = recuperer_bulletins_rss(alerte_url)

bulletins = avis_bulletins + alerte_bulletins

# Afficher les bulletins pour vérification
for bulletin in bulletins:
    print(bulletin)

{'Titre du bulletin (ANSSI)': 'Multiples vulnérabilités dans Mattermost Server (13 mai 2025)', 'Description': 'De multiples vulnérabilités ont été découvertes dans Mattermost Server. Elles permettent à un attaquant de provoquer une atteinte à la confidentialité des données et un contournement de la politique de sécurité.', 'Lien du bulletin (ANSSI)': 'https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0392/', 'Date de publication': 'Tue, 13 May 2025 00:00:00 +0000'}
{'Titre du bulletin (ANSSI)': 'Multiples vulnérabilités dans les produits SAP (13 mai 2025)', 'Description': "De multiples vulnérabilités ont été découvertes dans les produits SAP. Certaines d'entre elles permettent à un attaquant de provoquer une exécution de code arbitraire à distance, une atteinte à la confidentialité des données et une injection de code indirecte à distance (XSS).", 'Lien du bulletin (ANSSI)': 'https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0396/', 'Date de publication': 'Tue, 13 May 2025 00:00:00 +00

In [3]:
# Étape 2 : Extraction des CVE
import requests
import re

def extraire_cves_depuis_bulletin(lien_bulletin):
   
    if not lien_bulletin.endswith('/'):
        lien_bulletin += '/'
    url_json = lien_bulletin + "json/"
    response = requests.get(url_json)
    data = response.json()
    
    ref_cves = [cve.get("name") for cve in data.get("cves", [])]
    print("CVE référencés :", ref_cves)
    
    cve_pattern = r"CVE-\d{4}-\d{4,7}"
    cve_list = list(set(re.findall(cve_pattern, str(data))))
    print("CVE trouvés par regex :", cve_list)
    
    return ref_cves, cve_list

print("\n=== Extraction des CVE ===")
for bulletin in bulletins:
    ref_cves, cve_list = extraire_cves_depuis_bulletin(bulletin["Lien du bulletin (ANSSI)"])
    bulletin["CVEs"] = list(set(ref_cves + cve_list))
    print(f"Bulletin {bulletin['Titre du bulletin (ANSSI)']} : CVEs = {bulletin['CVEs']}")


=== Extraction des CVE ===
CVE référencés : ['CVE-2025-4128', 'CVE-2025-4573']
CVE trouvés par regex : ['CVE-2025-4573', 'CVE-2025-4128']
Bulletin Multiples vulnérabilités dans Mattermost Server (13 mai 2025) : CVEs = ['CVE-2025-4573', 'CVE-2025-4128']
CVE référencés : ['CVE-2025-43003', 'CVE-2025-43007', 'CVE-2025-23191', 'CVE-2025-42999', 'CVE-2025-43009', 'CVE-2025-43011', 'CVE-2025-43006', 'CVE-2025-0060', 'CVE-2025-30012', 'CVE-2025-43000', 'CVE-2025-43004', 'CVE-2025-31324', 'CVE-2025-43005', 'CVE-2025-43008', 'CVE-2025-31329', 'CVE-2025-30009', 'CVE-2025-30011', 'CVE-2025-43002', 'CVE-2025-26662', 'CVE-2025-30010', 'CVE-2025-42997', 'CVE-2025-0061', 'CVE-2025-43010', 'CVE-2024-39592', 'CVE-2025-30018']
CVE trouvés par regex : ['CVE-2025-31324', 'CVE-2025-0060', 'CVE-2025-31329', 'CVE-2025-23191', 'CVE-2025-26662', 'CVE-2025-43003', 'CVE-2025-30009', 'CVE-2025-30012', 'CVE-2025-43004', 'CVE-2025-42997', 'CVE-2025-43002', 'CVE-2025-30010', 'CVE-2025-43007', 'CVE-2025-0061', 'CVE-


=== Enrichissement des CVE ===
